In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
import matplotlib.image as mpimg
from keras import Sequential
from keras.layers import Conv2D,Dense,MaxPooling2D,BatchNormalization,Flatten,Dropout
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping

In [ ]:
base_dir, _ = os.path.splitext("../../../../Downloads/chest_xray")

In [ ]:
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

train_normal = os.path.join(train_dir, 'NORMAL')
print ('Total training normal images:', len(os.listdir(train_normal)))
train_pneu = os.path.join(train_dir, 'PNEUMONIA')
print ('Total training pneu images:', len(os.listdir(train_pneu)))

val_normal = os.path.join(validation_dir, 'NORMAL')
print ('Total validation normal images:', len(os.listdir(val_normal)))
val_pneu = os.path.join(validation_dir, 'PNEUMONIA')
print ('Total validation pneu images:', len(os.listdir(val_pneu)))

test_dir = os.path.join(base_dir,'test')
test_normal = os.path.join(test_dir, 'NORMAL')
print ('Total test normal images:', len(os.listdir(test_normal)))
test_pneu = os.path.join(test_dir, 'PNEUMONIA')
print ('Total test pneu images:', len(os.listdir(test_pneu)))

In [ ]:
image_size = 224 # All images will be resized to 160x160
batch_size = 8

# Rescale all images by 1./255 and apply image augmentation
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
                train_dir,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_directory(
                validation_dir, # Source directory for the validation images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
test_generator = test_datagen.flow_from_directory(
                test_dir, # Source directory for the validation images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='binary',
                shuffle=False)

In [ ]:
IMG_SHAPE = (image_size, image_size, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
print("Number of layers in the base model: ", len(base_model.layers))

In [ ]:
base_model.trainable = True

In [ ]:
# Fine tune from this layer onwards
fine_tune_at = 120

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

In [ ]:
model = tf.keras.Sequential([
  base_model,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dropout(.3),
  keras.layers.Dense(1, activation='sigmoid')
    
])

In [ ]:
# model.add()

In [ ]:
# model.compile(optimizer = Adam(learning_rate=0.01),
#               loss='binary_crossentropy',
#               metrics=['accuracy'])

In [ ]:
# epochs = 15
# steps_per_epoch = train_generator.n // batch_size
# validation_steps = validation_generator.n // batch_size

# history = model.fit(train_generator,
#                               steps_per_epoch = steps_per_epoch,
#                               epochs=epochs,
#                               workers=4,
#                               validation_data=validation_generator,
#                               validation_steps=validation_steps)

In [ ]:
model2 = Sequential()

In [ ]:
model2.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(image_size, image_size, 3)))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

model2.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

model2.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())


model2.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())
model2.add(Dropout(.3))


model2.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())
model2.add(Dropout(0.4))

model2.add(Flatten())
model2.add(Dense(128, activation='relu'))
# model2.add(Dropout(0.3))
model2.add(Dense(1, activation = 'sigmoid'))

In [ ]:
model2.compile(optimizer = SGD(learning_rate = .001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
epochs = 20
steps_per_epoch = train_generator.n // batch_size
validation_steps = test_generator.n // batch_size
# lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=2, mode='max')
es = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
cb_list = [es]
history2 = model2.fit(train_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs,
                              validation_data=test_generator,
                              validation_steps=validation_steps,
                              callbacks = cb_list )

In [ ]:
# model2.save('sgdmodel.h5')

In [ ]:
# test_generator.labels

In [ ]:
predictions = []
for x in model2.predict(test_generator):
    for z in x:
        predictions.append(np.round(z))
        
metrics.confusion_matrix(test_generator.classes,predictions)